In [2]:
import numpy as np
import time

## Q2: Histrograms

Here we will read in columns of numbers from a file and create a histogram, using NumPy routines.  Make sure you have the data file
"`sample.txt`" in the same directory as this notebook. You download it from  https://raw.githubusercontent.com/sbu-python-summer/python-tutorial/master/day-3/sample.txt (and use python to download a file!)

  * Use `np.loadtxt()` to read this file in.  

  * Next, use `np.histogram()` to create a histogram array.  The output returns both the count and an array of edges.
  
  * Finally, loop over the bins and print out the bin center (averaging the left and right edges of the bin) and the count for that bin.

In [2]:
import urllib.request

urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/sbu-python-summer/python-tutorial/master/day-3/sample.txt", "q2.txt"
)

('q2.txt', <http.client.HTTPMessage at 0x7f3bd0355f00>)

In [13]:
data = np.loadtxt("q2.txt", unpack=True)
vals, edges = np.histogram(data)
centers = (edges[:-1] + edges[1:]) / 2
for center, value in zip(centers, vals):
    print(f"{center:.2f} {value}")

-24.11 6
-11.15 23
1.81 52
14.77 37
27.73 16
40.69 14
53.64 13
66.60 13
79.56 13
92.52 13


## Q3: Are you faster than numpy?

Numpy of course has a standard deviation function, `np.std()`, but here we'll write our own that works on a 1-d array (vector).  The standard
deviation is a measure of the "width" of the distribution of numbers
in the vector.

Given an array, $a$, and an average $\bar{a}$, the standard deviation
is:
$$
\sigma = \left [ \frac{1}{N} \sum_{i=1}^N (a_i - \bar{a})^2 \right ]^{1/2}
$$

Write a function to calculate the standard deviation for an input array, `a`:

  * First compute the average of the elements in `a` to define $\bar{a}$
  * Next compute the sum over the squares of $a - \bar{a}$
  * Then divide the sum by the number of elements in the array
  * Finally take the square root (you can use `np.sqrt()`)
  
Test your function on a random array, and compare to the built-in `np.std()`. Check the runtime as well.

In [14]:
def f_std(a):
    s = 0
    for x in a:
        s += x
    mean = s / len(a)
    ssq = 0
    for x in a:
        ssq += (x - mean) ** 2
    return np.sqrt(ssq / len(a))

In [19]:
arr = np.random.rand(10**9)
t0 = time.time()
_ = np.std(arr)
t1 = time.time()
_ = f_std(arr)
t2 = time.time()
print(f"NP: {t1-t0}, python: {t2-t1}")

NP: 2.726123094558716, python: 244.11608934402466


## Q6: Conway's Game of Life

**Exercise**: Code up Conway's Game of Life using numpy 

The Game of Life is a cellular automaton devised by mathematician John Horton Conway in 1970. It is a zero-player game, meaning that its evolution is determined by its initial state, requiring no further input. One interacts with the Game of Life by creating an initial configuration and observing how it evolves. It is Turing complete and can simulate a universal constructor or any other Turing machine.

https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life

The Game of Life is *really* (really, really) cool. There are just four extremely simple rules, and these result in an immense richness of behaviour and complexity.

https://www.youtube.com/watch?v=C2vgICfQawE&t=221s&ab_channel=RationalAnimations

https://www.youtube.com/watch?v=jvSp6VHt_Pc&ab_channel=TheDevDoctor

Here some web apps to play:

https://conwaylife.com/

https://playgameoflife.com/

Some computational hints:

https://blog.datawrapper.de/game-of-life/

For instance, here is a Game-of-Life structure that sends a message at fixed intervals (that little spaceship leaving toward the bottom right)

![](https://blog.datawrapper.de/wp-content/uploads/2021/06/game-of-life-loop-cropped.gif)


In [171]:
def game_of_life(grid, N):
    """
    Runs the game of life for N iterations on the given grid. The grid can be a tuple of (rows, cols) in which case a random grid is generated.
    """
    if type(grid) == tuple and len(grid) == 2:
        grid = np.random.choice([1, 0], grid[0] * grid[1], p=[0.1, 0.9]).reshape(*grid)
    neighbor_filter = np.array([[1, 1, 1], [1, 0, 1], [1, 1, 1]])
    grids = [grid]
    for i in range(N):
        extended_grid = np.pad(grid, 1, mode="constant")
        neighbors = np.zeros_like(grid)
        for row in range(grid.shape[0]):
            for col in range(grid.shape[1]):
                neighbors[row, col] = np.sum(extended_grid[row : row + 3, col : col + 3] * neighbor_filter)
        newGrid = np.copy(grid)
        newGrid = np.where((grid == 1) & ((neighbors < 2) | (neighbors > 3)), 0, newGrid)
        newGrid = np.where((grid == 0) & (neighbors == 3), 1, newGrid)
        # if any(np.array_equal(newGrid, arr) for arr in grids):
        #     print("Reached a steady state at iteration", i + 1)
        # break
        grids.append(newGrid)
        grid = newGrid
    return grids

In [166]:
%%time
dims = (50, 50)
grid = np.random.choice([1, 0], dims[0] * dims[1], p=[0.1, 0.9]).reshape(*dims)
grids = game_of_life(grid, 100)

CPU times: user 1.16 s, sys: 69 µs, total: 1.16 s
Wall time: 1.16 s
